In [9]:
!pip install --upgrade pip
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: Ignored the following versions that require a different python version: 0.1.1 Requires-Python >=3.9; 0.1.2 Requires-Python >=3.8; 0.2.0 Requires-Python >=3.8; 0.3.0 Requires-Python >=3.8; 0.3.1 Requires-Python >=3.8; 0.3.2 Requires-Python >=3.8; 0.3.3 Requires-Python >=3.8; 0.4.0 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement tiktoken (from versions: none)
ERROR: No matchin

In [13]:
!pip install transformers==4.30.2
!pip install accelerate -U
!pip install sentencepiece -U

  Obtaining dependency information for transformers==4.30.2 from https://files.pythonhosted.org/packages/5b/0b/e45d26ccd28568013523e04f325432ea88a442b4e3020b757cf4361f0120/transformers-4.30.2-py3-none-any.whl.metadata
  Using cached transformers-4.30.2-py3-none-any.whl.metadata (113 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00:00:01
Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/10/d3/5382aa337d3e67214003a17b06bfc0

In [98]:
from transformers import AutoTokenizer

In [99]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [118]:
import os

In [119]:
def parse_faq(file_content,QA_SEP='====='):
    arr = file_content.split(QA_SEP)
    list_arr = []
    for item in arr:
        question, answer = item.strip().split("\n", 1)
        question = question.replace("Question: ", "")
        answer = answer.replace("Answer: ", "")
        list_arr.append((answer,question))
    return list_arr

filenames = os.listdir('topwar_faq')


all_data = []
for fn in filenames:
    if fn == '.ipynb_checkpoints':
        continue
    with open(f"topwar_faq/{fn}") as f:
        data = f.read()
        all_data += parse_faq(data)
print(f"data size:{len(all_data)}")

data size:685


In [102]:
import time

In [103]:
import os,json
import openai
openai.api_key = ''


def call_llm(prompt):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k-0613",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message['content']


In [104]:
import pickle
def save_file(data,cach_name='cach.pkl'):
    with open(cach_name,"wb") as f:
        pickle.dump(data,f)

In [105]:
import math,time
def generate_query(alldata,tokenizer):
    new_data = []
    for answer, query in alldata:
        context = f"问:{query}\n答:{answer}"
        tokens = len(tokenizer(context)['input_ids'])
        nums =  math.ceil(tokens/40)
        prompt= f"请根据以下内容生成{nums}个独立的问题\n ```\n{context}```"
        response = call_llm(prompt)
        print (f"tokens:{tokens},prompt:{prompt}\nresponse:{response}\n--------------")
        for query in response.split('\n'):
            new_data.append((context,answer,query))
        save_file(new_data)
        time.sleep(20)
        
    return new_data

In [ ]:
generate_query(all_data,tokenizer)

In [ ]:
newdata = None
with open('cach.pkl', 'rb') as f:
    newdata = pickle.load(f)

In [115]:
len(newdata)

1704

In [127]:
new_all_data = [ (a,q) for _,a,q in newdata]+all_data

In [128]:
len(new_all_data)

2389

In [130]:
with open('topwar_faq/jiangyu.pkl', 'wb') as f:
    pickle.dump(newdata,f)